In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.style.available

In [ ]:
# sns.heatmap(correlation_matrix,
#             annot=True,
#             linewidths=0.4,
#             annot_kws={"size": 10})

### import data

In [ ]:
appointments = pd.read_csv('Appointments.csv')

In [ ]:
calls = pd.read_csv('CallsRingCentral.csv')

In [ ]:
reason_for_visit = pd.read_csv('MeetingReasonForVisits.csv')

In [ ]:
meeting_status = pd.read_csv('MeetingStatus.csv')

In [ ]:
offices = pd.read_csv('Offices.csv')

In [ ]:
providers_schedules = pd.read_csv('ProvidersSchedulesLastest.csv')

### explore data

In [ ]:
appointments.info()

In [ ]:
appointments.head().T

In [ ]:
reason_for_visit.head()

In [ ]:
reason_for_visit.info()

In [ ]:
len(reason_for_visit['Name'].unique())

In [ ]:
meeting_status.head()

In [ ]:
meeting_status.info()

In [ ]:
offices.head()

In [ ]:
offices.info()

In [ ]:
providers_schedules.head()

In [ ]:
providers_schedules.info()

In [ ]:
calls.head().T

In [ ]:
calls.info()

### combine/merge dataframes

In [ ]:
merged1 = pd.merge(left=appointments, right=reason_for_visit, how='left', left_on='MeetingReasonForVisitId',\
                  right_on='Id')

In [ ]:
appointments.shape, reason_for_visit.shape, merged1.shape

In [ ]:
merged1 = merged1.rename(columns={'MeetingReasonForVisitId': 'ReasonForVisitId', 'Name':'ReasonForVisitName', 'Description':'ReasonForVisitDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
# merged1.head().T

In [ ]:
# merge in office name from offices df
merged1 = pd.merge(left=merged1, right=offices, how='left', left_on='OfficeId', right_on='id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'OfficeName', 'id_x':'id'})

In [ ]:
merged1.drop('id_y', axis=1, inplace=True)

In [ ]:
merged1 = pd.merge(left=merged1, right=meeting_status, how='left', left_on='MeetingStatusId', right_on='Id')

In [ ]:
merged1 = merged1.rename(columns={'Name':'MeetingStatusName', 'Description':'MeetingStatusDescription'})

In [ ]:
merged1.drop('Id', axis=1, inplace=True)

In [ ]:
merged1.info()

In [ ]:
merged1.head().T

In [ ]:
# rearrange column order to group releveant columns together
merged1.columns

In [ ]:
ordered_columns = ['id', 'Patient', 'PatientAgeMeetingDate', 'PatientGender',
       'PatientState', 'PatientCity', 'PatientInsurance', 'Provider',
       'Specialty', 'AppointmentDate', 'AppointmentCreated', 'AppointmentDuration', 'ReasonForVisitId', 'ReasonForVisitName',
       'ReasonForVisitDescription','MeetingStatusId', 'MeetingStatusName',
       'MeetingStatusDescription', 'OfficeId',  'OfficeName', 'CreatedBy']

In [ ]:
merged1 = merged1[ordered_columns]

In [ ]:
# remove time component from AppointmentDate and AppointmentCreated columns

In [ ]:
merged1['AppointmentCreated'] = merged1['AppointmentCreated'].str.slice(start=0, stop=10)

In [ ]:
merged1['AppointmentDate'] = merged1['AppointmentDate'].str.slice(start=0, stop=10)

In [ ]:
# merged1.head().T

In [ ]:
# pd.value_counts(merged1['PatientState'])

In [ ]:
# pd.value_counts(merged1['PatientGender'])

In [ ]:
# pd.value_counts(merged1['PatientCity'])

In [ ]:
# pd.value_counts(merged1['MeetingStatusName'])

In [ ]:
pd.value_counts(merged1['AppointmentDuration'])

In [ ]:
pd.value_counts(merged1['Specialty'])

In [ ]:
# convert date columns to datetime 
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate']).apply(lambda x: x.date()) #,format='%Y-%m-%d')

In [ ]:
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated']).apply(lambda x: x.date()) #, format='%Y-%m-%d')

In [ ]:
# calculate time between AppointmentCreated and AppointmentDate
merged1['DaysFromAppointmentCreatedToVisit'] = (merged1['AppointmentDate'] - merged1['AppointmentCreated']).dt.days

In [ ]:
merged1.head().T

In [ ]:
merged1.info()

In [ ]:
merged1['Specialty'].isnull()
merged1.isnull().sum()

In [ ]:
# pd.value_counts(merged1['DaysFromAppointmentCreatedToVisit'])

In [ ]:
merged1[merged1['DaysFromAppointmentCreatedToVisit'] < 0][['DaysFromAppointmentCreatedToVisit', 'AppointmentCreated', 'AppointmentDate']]

In [ ]:
merged1['AppointmentDate'] = pd.to_datetime(merged1['AppointmentDate'])
merged1['AppointmentCreated'] = pd.to_datetime(merged1['AppointmentCreated'])

In [ ]:
merged1.dtypes

In [ ]:
merged1.columns

In [ ]:
pd.value_counts(merged1['Specialty'])

In [ ]:
medical = merged1[merged1['Specialty'] == 'Medical']

In [ ]:
pd.value_counts(medical['ReasonForVisitName'])

In [ ]:
medical1 = medical.set_index('AppointmentDate')

In [ ]:
# medical1.groupby('ReasonForVisitName').agg({'ReasonForVisitName': 'count'})

In [ ]:
# medical1.groupby('ReasonForVisitId').agg({'ReasonForVisitId': 'count'})

In [ ]:
merged2 = merged1.set_index('AppointmentDate')

In [ ]:
merged_index_month = merged2.index.month

In [ ]:
merged_index_year = merged2.index.year

In [ ]:
merged2.index.date

In [ ]:
specialty_counts = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['Specialty'].count()

In [ ]:
specialty_counts = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['Specialty'].count()

In [ ]:
specialty_counts

In [ ]:
type(specialty_counts)

In [ ]:
specialty = merged2[['Specialty']].dropna(axis=0)

In [ ]:
specialty.isnull().sum()

In [ ]:
a_index_year = specialty.index.year
a_index_month = specialty.index.month

In [ ]:
a_index_month

In [ ]:
specialty_df = pd.DataFrame(specialty.groupby([a_index_year, a_index_month, 'Specialty'])['Specialty'].count().unstack('Specialty'))

In [ ]:
specialty_df

In [ ]:
specialty_bfilled = specialty_df.fillna(method='bfill')

In [ ]:
cols = list(specialty_df.columns)
means = specialty_df[cols].mean()
means.index

In [ ]:
# fill missing values with column mean
for idx in means.index:
    specialty_df[idx].fillna(means[idx], inplace=True)

In [ ]:
specialty_df

In [ ]:
# using bfill to fill NaN values
ax5 = specialty_bfilled.plot(figsize=(17,14), colormap='Dark2')
ax5.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax5.set_xlabel('Date', fontsize=25)
ax5.set_ylabel('Number of Appointments', fontsize=25)
ax5.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# if NaNs are filled by column mean
ax4 = specialty_df.plot(figsize=(17,14), colormap='Dark2')
ax4.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax4.set_xlabel('Date', fontsize=25)
ax4.set_ylabel('Number of Appointments', fontsize=25)
ax4.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# plot number of appoints by specialty for each month, adding multiple years
ax3 = specialty.groupby([a_index_year, a_index_month, 'Specialty'])['Specialty'].count().unstack('Specialty').plot(figsize=(17,15),\
                                                    colormap='Dark2')
ax3.set_ylim(0, 1200)
ax3.set_xlim(0,28)
ax3.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax3.set_xlabel('Date', fontsize=25)
ax3.set_ylabel('Number of Appointments', fontsize=25)
ax3.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# plot number of appoints by specialty for each month, adding multiple years
ax2 = specialty.groupby([a_index_year, a_index_month, 'Specialty'])['Specialty'].count().unstack('Specialty').plot.area(figsize=(17,15),\
                                                    colormap='Dark2')
ax2.set_ylim(0, 3000)
ax2.set_xlim(0,28)
ax2.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax2.set_xlabel('Date', fontsize=25)
ax2.set_ylabel('Number of Appointments', fontsize=25)
ax2.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# plot number of appoints by specialty for each month, adding months from multiple years together
ax = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot(figsize=(17,15),\
                                                    colormap='Dark2')
ax.set_ylim(0, 1200)
ax.set_xlim(0,13)
ax.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax.set_xlabel('Month', fontsize=25)
ax.set_ylabel('Number of Appointments', fontsize=25)
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# plot number of appoints by specialty for each month, adding months from multiple years together
ax1 = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot.area(figsize=(20,15),\
                                                                                                colormap='Accent')
ax1.set_ylim(0, 5000)
ax1.set_xlim(1, 12)
ax1.set_title('Number of Appointments per Month by Specialty', fontsize=30)
ax1.set_xlabel('Month', fontsize=25)
ax1.set_ylabel('Number of Appointments', fontsize=25)
ax1.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
ax = merged2.groupby([merged_index_month, 'Specialty'])['Specialty'].count().unstack().plot(subplots=True,\
                        colormap='Dark2', figsize=(20,15), layout=(5,2), sharex=False, sharey=False)
plt.xlabel('Month')
plt.tight_layout()
plt.show()

In [ ]:
merged2.head(2).T

In [ ]:
time_spent_by_month = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['AppointmentDuration'].sum()

In [ ]:
time_spent_by_month

In [ ]:
merged2.groupby([merged_index_year, merged_index_month])['AppointmentDuration'].sum().plot(figsize=(10,6))
plt.xlabel('Date')
plt.ylabel('Time (min)')
plt.title('Sum of time spent for all Specialties together')
plt.show()

In [ ]:
ax6 = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['AppointmentDuration'].sum().unstack().plot(figsize=(17,15),\
                                                    colormap='Dark2', linewidth=3)
ax6.set_ylim(0, 32000)
ax6.set_xlim(0,25)
ax6.set_title('Time per Month by Specialty', fontsize=30)
ax6.set_xlabel('Date', fontsize=25)
ax6.set_ylabel('Time (minutes)', fontsize=25)
ax6.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
# Area plot of time spent for each specialty
ax7 = merged2.groupby([merged_index_year, merged_index_month, 'Specialty'])['AppointmentDuration']\
.sum().unstack().plot.area(figsize=(17,15), colormap='Dark2', linewidth=3)
                                                   
ax7.set_ylim(0, 130000)
ax7.set_xlim(0,25)
ax7.set_title('Time per Month by Specialty', fontsize=30)
ax7.set_xlabel('Date', fontsize=25)
ax7.set_ylabel('Time (minutes)', fontsize=25)
ax7.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
date = merged2.index
merged2.groupby([date,'Specialty'])['AppointmentDuration'].sum()

In [ ]:
date = pd.to_datetime(merged2.index,format='%Y-%m-%d')

In [ ]:
date
merged2['AppointmentDate'] = date

In [ ]:
merged2.head().T

In [ ]:
type(merged2['AppointmentDate'])

In [ ]:
merged2.groupby(['AppointmentDate', 'Specialty'])['AppointmentDuration'].sum()

In [ ]:
date = pd.to_datetime(merged2.index)
ax8 = merged2.groupby(['AppointmentDate', 'Specialty'])['AppointmentDuration'].sum().unstack().\
plot.area(figsize=(30,30),colormap='Dark2', linewidth=3)
                                                    
# ax8.set_ylim(0, 32000)
# ax8.set_xlim(0,25)
ax8.set_title('Time per Month by Specialty', fontsize=30)
ax8.set_xlabel('Date', fontsize=25)
ax8.set_ylabel('Time (minutes)', fontsize=25)
# ax8.legend(loc='center left', bbox_to_anchor=(1.0, 0.5), fontsize=20)
plt.show()

In [ ]:
plt.rcParams.keys()

In [ ]:
params = {'legend.fontsize': 20, 'legend.handlelength': 2, 'axes.labelsize': 'medium'}
plt.rcParams.update(params)
fig = merged2.groupby(['AppointmentDate', 'Specialty'])['AppointmentDuration'].sum().unstack().plot.area(subplots=True,\
    colormap='Dark2', figsize=(20,60), layout=(10,1), sharex=True, sharey=True, linewidth=3, fontsize=20)
plt.xlabel('Date')
plt.ylabel('Time (minutes)', fontsize=20)
plt.tight_layout()
plt.show()

In [ ]:
medical1['id'].groupby(medical1.index.month).count()

In [ ]:
medical1.columns

In [ ]:
index_month = medical1.index.month
counts = medical1['id'].groupby(index_month).count()
ax = counts.plot()
ax.set_ylim(0, 1200)
ax.set_xlim(0, 13)
ax.set_title('Number of Appointments per Month (Medical)', fontsize=15)
ax.set_xlabel('Month', fontsize=12)
ax.set_ylabel('Number of Appointments', fontsize=12)
plt.show()

In [ ]:
medical1.info()

In [ ]:
number_cols = ['Provider', 'ReasonForVisitId', 'MeetingStatusId', 'OfficeId', 'DaysFromAppointmentCreatedToVisit']
nummeric = merged2[number_cols]

In [ ]:
nummeric.head()

In [ ]:
days_until_appointment = nummeric['DaysFromAppointmentCreatedToVisit']

In [ ]:
days_until_appointment.head()

In [ ]:
# ax = days_until_appointment.plot(x=days_until_appointment.index, kind='hist', figsize=(12,6))
# ax.set_xlabel('Appointment Date', fontsize=16)
# # ax.set_ylabel('Count of each visit type')
# plt.show()

In [ ]:
nummeric.index